 # Pipeline ShapeWorks - Version Optimisée



 Dans cette version, **le grooming est effectué une seule fois** pour l'ensemble des données.

 Ensuite, pour chaque jeu de paramètres variables (dans la grille de `GRID_PARAMS`), **nous ré-exécutons uniquement les étapes d'optimisation, d'analyse PCA et de calcul de métriques**, en réutilisant les résultats du grooming déjà sauvegardés sur disque.



 ---



 ## Changements Demandés

 1. **Réorganiser le code** pour éviter la redondance entre pipeline "avec grooming" et "sans grooming".

    - On conserve des fonctions de *pré-traitement* (acquisition, grooming, rigid...) et des fonctions d'*optimisation / analyse*.

    - On n'a plus deux grosses fonctions pipeline distinctes, mais plutôt un flux clair :

      1) `run_preprocessing()` (acquire_data + groom_shapes + rigid_transformations)

      2) Pour chaque paramètre : `run_optimization_and_analysis()` (optimize + PCA + metrics)



 2. **Format des temps** dans l'Excel : au lieu de minutes entières, on veut `minutes:secondes`, pour voir également les petites durées.

 3. **Ajouter 3 colonnes** supplémentaires dans la feuille "PARAMS" (le tableau principal) :

    - `compactness_95` : nombre de composantes nécessaires pour atteindre 95% de la variance

    - `final_specificity_error` : la dernière valeur de l'erreur de spécificité

    - `final_generalization_error` : la dernière valeur de l'erreur de généralisation



 Le reste du code doit rester **identique** dans son fonctionnement et ses étapes, car la pipeline initiale « fonctionne parfaitement ».

 ## 1. Imports et Fonctions Utilitaires

In [37]:
import os
import shutil
import joblib
from pathlib import Path
import glob
import subprocess
import time
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.decomposition import PCA
import shapeworks as sw

# Pour tracer et insérer des images dans Excel
import matplotlib.pyplot as plt
from openpyxl.drawing.image import Image as ExcelImage
import tempfile
from openpyxl.styles import Alignment



 ### 0. Paramètres Généraux

In [38]:
# ---------- Dossiers ----------
DATASET_NAME       = "TEST"
DATASET_PATHS      = [
    # (os.path.join(".", "DATA", "RF_FULGUR_M"), "RF"),
    (os.path.join(".", "DATA", "RF_FULGUR_SAMPLE_2"), "RF_S2") 
]
SHAPE_EXT          = ".nii.gz"

# Sorties dans "./OUTPUT_PIPELINE"
BASE_OUTPUT_DIR    = os.path.abspath(os.path.join(".", "OUTPUT_PIPELINE"))
OPTION_SET         = "default"  # Pour nommer les fichiers

# ---------- Paramètres Grooming ----------
ANTIALIAS_ITERATIONS = 30
ISO_SPACING          = [1, 1, 1]
PAD_SIZE             = 10
PAD_VALUE            = 0
ISO_VALUE            = 0.5
ICP_ITERATIONS       = 100

# ---------- Paramètres par Défaut ----------
OPT_PARAMS = {
    "number_of_particles":       128,
    "use_normals":               0,
    "normals_strength":          10.0,
    "checkpointing_interval":    1000,
    "keep_checkpoints":          0,
    "iterations_per_split":      1000,
    "optimization_iterations":   1000,
    "starting_regularization":   10,
    "ending_regularization":     1,
    "relative_weighting":        1,
    "initial_relative_weighting":0.05,
    "procrustes_interval":       0,
    "procrustes_scaling":        0,
    "save_init_splits":          0,
    "verbosity":                 0,
    "multiscale":                1,
    "multiscale_particles":      32,
    "tiny_test":                 False,
    "use_single_scale":          0,
    "mesh_mode":                 True,  # True => optimisation sur des meshes
    "option_set":                OPTION_SET
}

# ---------- Paramètres Variables (Grid) ----------
GRID_PARAMS = {
    "number_of_particles": [16, 32, 64, 128]
}



In [39]:
# ---------- Préparation du dossier de sortie ----------
OUTPUT_DB = os.path.abspath(os.path.join(".", "OUTPUT_DB"))
os.makedirs(OUTPUT_DB, exist_ok=True)

if os.path.exists(BASE_OUTPUT_DIR):
    # Déplacer les fichiers Excel pré-existants
    for root, dirs, files in os.walk(BASE_OUTPUT_DIR):
        for file in files:
            if file.endswith(".xlsx"):
                source_file = os.path.join(root, file)
                new_filename = f"{os.path.splitext(file)[0]}_{int(time.time())}.xlsx"
                dest_file = os.path.join(OUTPUT_DB, new_filename)
                shutil.move(source_file, dest_file)
    # Supprimer le dossier complet pour repartir propre
    shutil.rmtree(BASE_OUTPUT_DIR)



 ### 1. Fonctions : Acquisition, Grooming, Rigid

In [40]:
def acquire_data(dataset_paths, shape_ext, output_path):
    """
    Récupère et trie les noms de fichiers .nii.gz
    Renvoie (shape_filenames, dataset_ids)
    """
    print("\n----------------------------------------")
    print("Step 1. Acquire Data")
    os.makedirs(output_path, exist_ok=True)

    shape_filenames = []
    dataset_ids = []

    for data_path, dataset_id in dataset_paths:
        files = sorted(glob.glob(os.path.join(data_path, '*' + shape_ext)))
        shape_filenames.extend(files)
        dataset_ids.extend([dataset_id] * len(files))

    print(f"Nombre de shapes : {len(shape_filenames)}")
    return shape_filenames, dataset_ids


def groom_shapes(shape_filenames, dataset_ids, groom_dir):
    """
    1) Crop bounding box
    2) Antialias + resample + binarize
    3) Pad
    """
    print("\n----------------------------------------")
    print("Step 2. Groom - Data Pre-processing")

    os.makedirs(groom_dir, exist_ok=True)

    shape_seg_list = []
    shape_names = []

    for i, shape_filename in enumerate(tqdm(shape_filenames, desc="Grooming shapes")):
        dataset_id = dataset_ids[i]
        base_shape_name = os.path.basename(shape_filename).replace(SHAPE_EXT, '')
        shape_name = f"{dataset_id}_{base_shape_name}"
        shape_names.append(shape_name)

        shape_seg = sw.Image(shape_filename)
        shape_seg_list.append(shape_seg)

        bounding_box = sw.ImageUtils.boundingBox([shape_seg], ISO_VALUE).pad(2)
        shape_seg.crop(bounding_box)

        shape_seg.antialias(ANTIALIAS_ITERATIONS).resample(ISO_SPACING, sw.InterpolationType.Linear).binarize()
        shape_seg.pad(PAD_SIZE, PAD_VALUE)

    return shape_seg_list, shape_names


def rigid_transformations(shape_seg_list, shape_names, groom_dir):
    """
    1) Trouve l'image de référence
    2) Calcule la rigid transform
    3) Convertit en distance transform (antialias->DT->blur)
    4) Sauvegarde en .nrrd
    """
    print("\n----------------------------------------")
    print("Step 3. Groom - Rigid Transformations")

    ref_index = sw.find_reference_image_index(shape_seg_list)
    ref_seg = shape_seg_list[ref_index]
    ref_name = shape_names[ref_index]

    os.makedirs(groom_dir, exist_ok=True)

    ref_filename = os.path.join(groom_dir, 'reference.nrrd')
    ref_seg.write(ref_filename)
    print(f"Image de référence trouvée : {ref_name}")

    transform_dir = os.path.join(groom_dir, 'rigid_transforms')
    os.makedirs(transform_dir, exist_ok=True)

    rigid_transforms = []

    for shape_seg, shape_name in tqdm(zip(shape_seg_list, shape_names),
                                      desc="Calcul des transformations",
                                      total=len(shape_seg_list)):
        rigid_transform = shape_seg.createRigidRegistrationTransform(ref_seg, ISO_VALUE, ICP_ITERATIONS)
        rigid_transform = sw.utils.getVTKtransform(rigid_transform)
        rigid_transforms.append(rigid_transform)

        transform_filename = os.path.join(transform_dir, f'{shape_name}_to_{ref_name}_transform.txt')
        np.savetxt(transform_filename, rigid_transform)

        shape_seg.antialias(ANTIALIAS_ITERATIONS).computeDT(0).gaussianBlur(1.5)

    # Sauvegarde finale en .nrrd
    output_subdir = 'distance_transforms'
    output_extension = '.nrrd'
    output_dir = os.path.join(groom_dir, output_subdir)
    os.makedirs(output_dir, exist_ok=True)

    groomed_files = []
    for shape_seg, shape_name in zip(shape_seg_list, shape_names):
        out_name = os.path.join(output_dir, shape_name + output_extension)
        shape_seg.write(out_name)
        groomed_files.append(out_name)

    return rigid_transforms, groomed_files


def run_preprocessing(dataset_paths, shape_ext, base_dir):
    """
    Orchestrateur pour la partie 'grooming':
      - acquisition
      - grooming
      - rigid
    Retourne : shape_seg_list, shape_filenames, dataset_ids, shape_names, rigid_transforms, groomed_files
    """
    output_path = os.path.join(base_dir, "OUTPUT")
    os.makedirs(output_path, exist_ok=True)
    groom_dir = os.path.join(output_path, "groomed")

    shape_filenames, dataset_ids = acquire_data(dataset_paths, shape_ext, output_path)
    shape_seg_list, shape_names = groom_shapes(shape_filenames, dataset_ids, groom_dir)
    rigid_transforms, groomed_files = rigid_transformations(shape_seg_list, shape_names, groom_dir)

    return shape_seg_list, shape_filenames, dataset_ids, shape_names, rigid_transforms, groomed_files



 ### 2. Fonctions : Optimisation, PCA, Métriques

In [41]:
def optimize_particles(shape_seg_list, shape_filenames, rigid_transforms, groomed_files, output_path):
    """
    Step 4 : Optimisation
    """
    print("\n----------------------------------------")
    print("Step 4. Optimize - Particle Based Optimization")

    os.makedirs(output_path, exist_ok=True)

    # -- Récupération domain_type + conversion si mesh_mode --
    domain_type, groomed_files_out = sw.data.get_optimize_input(
        groomed_files,
        OPT_PARAMS["mesh_mode"]
    )

    # Construction sujets
    subjects = []
    for i in range(len(shape_seg_list)):
        subj = sw.Subject()
        subj.set_number_of_domains(1)

        subj.set_original_filenames([os.path.abspath(shape_filenames[i])])
        subj.set_groomed_filenames([os.path.abspath(groomed_files_out[i])])
        subj.set_groomed_transforms([rigid_transforms[i].flatten()])

        try:
            subj.set_domain_type(0, domain_type)
        except AttributeError:
            pass

        subjects.append(subj)

    # Projet
    project = sw.Project()
    project.set_subjects(subjects)
    parameters = sw.Parameters()

    # Paramètres
    valid_params = {
        "number_of_particles":        OPT_PARAMS["number_of_particles"],
        "use_normals":                OPT_PARAMS["use_normals"],
        "normals_strength":           OPT_PARAMS["normals_strength"],
        "checkpointing_interval":     OPT_PARAMS["checkpointing_interval"],
        "keep_checkpoints":           OPT_PARAMS["keep_checkpoints"],
        "iterations_per_split":       OPT_PARAMS["iterations_per_split"],
        "optimization_iterations":    OPT_PARAMS["optimization_iterations"],
        "starting_regularization":    OPT_PARAMS["starting_regularization"],
        "ending_regularization":      OPT_PARAMS["ending_regularization"],
        "relative_weighting":         OPT_PARAMS["relative_weighting"],
        "initial_relative_weighting": OPT_PARAMS["initial_relative_weighting"],
        "procrustes_interval":        OPT_PARAMS["procrustes_interval"],
        "procrustes_scaling":         OPT_PARAMS["procrustes_scaling"],
        "save_init_splits":           OPT_PARAMS["save_init_splits"],
        "verbosity":                  OPT_PARAMS["verbosity"]
    }

    # tiny_test
    if OPT_PARAMS.get("tiny_test", False):
        valid_params["number_of_particles"] = 32
        valid_params["optimization_iterations"] = 25

    # multiscale
    if not OPT_PARAMS.get("use_single_scale", 0):
        valid_params["multiscale"] = 1
        valid_params["multiscale_particles"] = OPT_PARAMS["multiscale_particles"]

    # Remplissage
    for k, v in valid_params.items():
        parameters.set(k, sw.Variant([v]))

    project.set_parameters("optimize", parameters)

    # Sauvegarde .swproj
    proj_file = os.path.join(output_path, f"{DATASET_NAME}_{OPTION_SET}.swproj")
    project.save(proj_file)

    # Lancement de l'optim
    print("Lancement de l'optimisation via shapeworks...")
    cmd = ['shapeworks', 'optimize', '--progress', '--name', proj_file]
    subprocess.check_call(cmd, cwd=output_path)

    # Verification
    args_for_check = type('ArgsForCheck', (object,), {})()
    args_for_check.tiny_test = OPT_PARAMS.get("tiny_test", False)
    args_for_check.verify    = False
    sw.utils.check_results(args_for_check, proj_file)

    return os.path.join(output_path, f"{DATASET_NAME}_default_particles")


def get_particles(particles_dir, particle_type="world"):
    """
    Lit tous les fichiers *.world.particles
    Retourne un tableau [n_shapes, n_points, 3] et la liste des noms.
    """
    particles = []
    names = []
    for filename in os.listdir(particles_dir):
        if filename.endswith(particle_type + ".particles"):
            data = np.loadtxt(os.path.join(particles_dir, filename))
            particles.append(data)
            names.append(os.path.splitext(filename)[0])
    if not particles:
        return None, None
    return np.array(particles), names


def compute_pca(particles_dir, pca_output_dir):
    """
    Step 5 : PCA
    """
    print("\n----------------------------------------")
    print("Calcul de l'ACP")
    os.makedirs(pca_output_dir, exist_ok=True)

    parts, names = get_particles(particles_dir, "world")
    if parts is None:
        raise ValueError("Aucune particule chargée depuis " + particles_dir)

    n, p, _ = parts.shape
    parts_flat = parts.reshape(n, -1)
    print(f"Forme des particules aplaties : {parts_flat.shape}")

    pca = PCA(n_components=n - 1)
    pca.fit(parts_flat)
    comps = pca.transform(parts_flat)

    # Eigenvalues
    eigvals = pca.explained_variance_
    with open(os.path.join(pca_output_dir, 'eigenvalues.eval'), 'w') as f:
        for ev in eigvals:
            f.write(f"{ev}\n")

    # Eigenvectors
    eigenvectors = pca.components_
    eigenvectors_reshaped = eigenvectors.reshape(eigenvectors.shape[0], -1, 3)
    for i, eigvec in enumerate(eigenvectors_reshaped):
        fn = os.path.join(pca_output_dir, f"eigenvector_{i+1}.eig")
        np.savetxt(fn, eigvec, fmt='%f')

    # Projection sur 2 composantes
    pca_projection = comps[:, :2]

    print("Résultats de la PCA sauvegardés dans", pca_output_dir)
    return pca_projection, eigvals


def load_shapes(particles_dir, particle_type="world"):
    shapes = []
    for filename in os.listdir(particles_dir):
        if filename.endswith(particle_type + '.particles'):
            shape = np.loadtxt(os.path.join(particles_dir, filename))
            shapes.append(shape)
    return np.array(shapes)


def compute_compactness(eigenvalues, threshold=0.95):
    total_var = np.sum(eigenvalues)
    cum_var = np.cumsum(eigenvalues) / total_var
    num_comp = int(np.argmax(cum_var >= threshold) + 1)
    return num_comp, cum_var


def compute_specificity(real_shapes, num_particles, num_samples=1000):
    n, p, dim3 = real_shapes.shape
    d = p * dim3
    real_shapes_2d = real_shapes.reshape(n, d)

    Y = real_shapes_2d.T
    mu = np.mean(Y, axis=1, keepdims=True)
    Yc = Y - mu
    U, S, _ = np.linalg.svd(Yc, full_matrices=False)
    if S[0] < S[-1]:
        S = S[::-1]
        U = np.fliplr(U)

    specifics = np.zeros(n - 1)

    def shape_distance(ptsA, ptsB, pcount):
        A3 = ptsA.reshape(pcount, 3)
        B3 = ptsB.reshape(pcount, 3)
        return np.linalg.norm(A3 - B3, axis=1).sum()

    for m in tqdm(range(1, n), desc="Calcul de la spécificité"):
        epsi = U[:, :m]
        stdevs = np.sqrt(S[:m])
        betas = np.random.randn(m, num_samples)
        for i_mode in range(m):
            betas[i_mode, :] *= stdevs[i_mode]
        synth = epsi @ betas + mu
        min_dists = np.zeros(num_samples)
        for isyn in range(num_samples):
            sy = synth[:, isyn]
            best = 1e15
            for j in range(n):
                dist_j = shape_distance(sy, Y[:, j], num_particles)
                if dist_j < best:
                    best = dist_j
            min_dists[isyn] = best
        specifics[m-1] = np.mean(min_dists) / float(num_particles)

    return specifics


def compute_generalization(real_shapes, num_particles):
    if len(real_shapes.shape) == 3 and real_shapes.shape[2] == 3:
        n, p, dim3 = real_shapes.shape
        d = p * dim3
        real_shapes_2d = real_shapes.reshape(n, d)
    else:
        n, d = real_shapes.shape
        real_shapes_2d = real_shapes

    def shape_distance(ptsA, ptsB, pcount):
        A3 = ptsA.reshape(pcount, 3)
        B3 = ptsB.reshape(pcount, 3)
        return np.linalg.norm(A3 - B3, axis=1).sum()

    P = real_shapes_2d.T
    gens = np.zeros(n - 1)

    for m in range(1, n):
        tot_dist = 0.0
        for leave in range(n):
            Y = np.zeros((P.shape[0], n-1))
            Y[:, :leave] = P[:, :leave]
            Y[:, leave:] = P[:, leave+1:]
            mu = np.mean(Y, axis=1, keepdims=True)
            Yc = Y - mu
            U, S, _ = np.linalg.svd(Yc, full_matrices=False)
            epsi = U[:, :m]

            ytest = P[:, leave:leave+1]
            betas = epsi.T @ (ytest - mu)
            rec = epsi @ betas + mu

            dist = shape_distance(rec, ytest, num_particles) / float(num_particles)
            tot_dist += dist
        gens[m - 1] = tot_dist / float(n)

    return gens


def compute_error_metrics(particles_dir, pca_output_dir, num_particles):
    """
    Steps 5 & 6 : on calcule 3 métriques (compactness, specificity, generalization).
    """
    print("\n----------------------------------------")
    print("Calcul des métriques d'erreur")

    real_shapes = load_shapes(particles_dir, "world")
    if real_shapes.size == 0:
        raise ValueError(f"Aucune shape chargée dans {particles_dir}")

    eigenvalues_path = os.path.join(pca_output_dir, 'eigenvalues.eval')
    if not os.path.exists(eigenvalues_path):
        raise FileNotFoundError("Fichier eigenvalues.eval introuvable : " + eigenvalues_path)
    eigenvalues = np.loadtxt(eigenvalues_path)

    # 1) Compactness
    c_required, c_variance = compute_compactness(eigenvalues)

    # 2) Specificity
    specifics = compute_specificity(real_shapes, num_particles)

    # 3) Generalization
    generals = compute_generalization(real_shapes, num_particles)

    metrics = {
        "compactness_required": c_required,
        "cumulative_variance": c_variance.tolist(),
        "specificity": specifics.tolist(),
        "generalization": generals.tolist()
    }
    return metrics



 ### 3. Fonction Principale : Optim + PCA + Métriques pour un set de paramètres

In [42]:
def run_optimization_and_analysis(run_params, run_index,
                                  shape_seg_list, shape_filenames,
                                  rigid_transforms, groomed_files,
                                  base_output_dir):
    """
    Etapes :
      - Optimisation
      - PCA
      - Metrics
    """
    print("\n========================================")
    print(f"Lancement Optim+Analyse - Exécution {run_index}")

    overall_start = time.time()

    # Sous-dossier pour ce run
    run_dir = os.path.join(base_output_dir, f"Run_{run_index}")
    os.makedirs(run_dir, exist_ok=True)
    output_path = os.path.join(run_dir, "OUTPUT")
    os.makedirs(output_path, exist_ok=True)
    pca_out = os.path.join(output_path, "PCA_results")

    # MàJ des OPT_PARAMS
    OPT_PARAMS.update(run_params)

    step_times = {}

    # -- Step 4 : optimize --
    t0 = time.time()
    particles_dir = optimize_particles(
        shape_seg_list,
        shape_filenames,
        rigid_transforms,
        groomed_files,
        output_path
    )
    step_times["optimization"] = time.time() - t0

    # -- Step 5 : PCA --
    t0 = time.time()
    pca_projection, eigenvalues = compute_pca(particles_dir, pca_out)
    step_times["pca"] = time.time() - t0

    # -- Step 6 : metrics --
    t0 = time.time()
    n_parts = OPT_PARAMS.get("number_of_particles", 128)
    metrics = compute_error_metrics(particles_dir, pca_out, n_parts)
    step_times["error_metrics"] = time.time() - t0

    overall_time = time.time() - overall_start
    print(f"Exécution {run_index} terminée en {overall_time:.2f} secondes")

    return {
        "pca_projection": pca_projection,
        "metrics": metrics,
        "params": run_params,
        "step_times": step_times,
        "total_execution_time": overall_time
    }



 ### 4. Sauvegarde du fichier Excel

In [ ]:
def mm_ss_format(seconds):
    mm = int(seconds // 60)
    ss = int(seconds % 60)
    return f"{mm}:{ss:02d}"


def _plot_metric_curve(data, title, ylabel, run_idx, outdir, figsize=(5, 3)):
    os.makedirs(outdir, exist_ok=True)
    fn = f"{title.replace(' ', '_')}_Run_{run_idx}.png"
    image_path = os.path.join(outdir, fn)

    fig, ax = plt.subplots(figsize=figsize)
    ax.plot(range(1, len(data) + 1), data, marker='o')
    ax.set_title(title)
    ax.set_xlabel("Number of Modes")
    ax.set_ylabel(ylabel)
    ax.grid(True)

    plt.savefig(image_path, dpi=130, bbox_inches="tight")
    plt.close(fig)

    return image_path


def save_results_to_excel(results, excel_filename, grid_keys):
    """
    Génère le fichier Excel final.
    - Feuille "PARAMS" : 
        * run index
        * grid_keys
        * time_grooming, time_rigid, time_optimization, time_total (au format mm:ss)
        * + 3 colonnes : compactness_95, final_specificity_error, final_generalization_error
        * plus, en bas, la liste multi-ligne des paramètres
    - Feuilles "Run_i" :
        * Graphes (Compactness, Specificity, Generalization)
        * Valeurs PC1/PC2
    """
    print("\nSauvegarde des résultats dans le fichier Excel...")

    # Colonnes supplémentaires pour les 3 métriques demandées
    table_cols = (
        ["Run Index"] + list(grid_keys) +
        ["time_grooming", "time_rigid", "time_optimization", "time_total",
         "compactness_95", "final_specificity_error", "final_generalization_error"]
    )

    data_for_df = []

    for i, res in enumerate(results, start=1):
        row_dict = {}
        row_dict["Run Index"] = i

        # Paramètres variables
        for k in grid_keys:
            row_dict[k] = res["params"].get(k, None)

        # On n'a groomé qu'une seule fois ; la pipeline d'optim n'a pas de grooming/rigid
        # => On peut mettre 0 pour grooming, rigid,
        #    ou si vous aviez mesuré ces temps, vous pourriez les stocker. 
        #    Ici on laisse 0. 
        st = res["step_times"]
        row_dict["time_grooming"]      = "0:00"
        row_dict["time_rigid"]         = "0:00"
        row_dict["time_optimization"]  = mm_ss_format(st.get("optimization", 0))
        row_dict["time_total"]         = mm_ss_format(res.get("total_execution_time", 0))

        # Ajout des 3 métriques dans la feuille "PARAMS"
        # (on prend la dernière valeur en specificity / generalization)
        met = res["metrics"]
        row_dict["compactness_95"]            = met["compactness_required"]
        row_dict["final_specificity_error"]   = ( met["specificity"][-1] 
                                                  if len(met["specificity"])>0 else None )
        row_dict["final_generalization_error"]= ( met["generalization"][-1] 
                                                  if len(met["generalization"])>0 else None )

        data_for_df.append(row_dict)

    # DataFrame final
    params_df = pd.DataFrame(data_for_df, columns=table_cols)

    # Ouverture du writer
    writer = pd.ExcelWriter(excel_filename, engine='openpyxl')
    params_df.to_excel(writer, sheet_name="PARAMS", index=False)

    # Ajustement colonnes
    ws_params = writer.book["PARAMS"]
    col_widths = {
        "A":15, "B":15, "C":15, "D":15, "E":15,
        "F":15, "G":15, "H":15, "I":15, "J":20, "K":25, "L":25
    }
    for col, w in col_widths.items():
        ws_params.column_dimensions[col].width = w

    # Ajout du bloc paramètre complet en I–J (ligne = len(results)+3)
    row_for_params = len(results) + 3

    grooming_keys = [
        "ANTIALIAS_ITERATIONS", "ISO_SPACING", "PAD_SIZE",
        "PAD_VALUE", "ISO_VALUE", "ICP_ITERATIONS"
    ]
    all_param_names = sorted(set(OPT_PARAMS.keys()) | set(grooming_keys))

    param_names_list  = []
    param_values_list = []
    for pname in all_param_names:
        if pname in OPT_PARAMS:
            val = OPT_PARAMS[pname]
        else:
            if pname == "ANTIALIAS_ITERATIONS":
                val = ANTIALIAS_ITERATIONS
            elif pname == "ISO_SPACING":
                val = str(ISO_SPACING)
            elif pname == "PAD_SIZE":
                val = PAD_SIZE
            elif pname == "PAD_VALUE":
                val = PAD_VALUE
            elif pname == "ISO_VALUE":
                val = ISO_VALUE
            elif pname == "ICP_ITERATIONS":
                val = ICP_ITERATIONS
            else:
                val = None
        param_names_list.append(str(pname))
        param_values_list.append(str(val))

    c_names  = ws_params.cell(row=row_for_params, column=10)  # J
    c_values = ws_params.cell(row=row_for_params, column=11)  # K

    c_names.value  = "\n".join(param_names_list)
    c_values.value = "\n".join(param_values_list)

    c_names.alignment  = Alignment(wrap_text=True)
    c_values.alignment = Alignment(wrap_text=True)


    # Feuilles "Run_i" (graphes + PC1/PC2)
    for i, res in enumerate(results, start=1):
        sheet_name = f"Run_{i}"
        df_dummy = pd.DataFrame()
        df_dummy.to_excel(writer, sheet_name=sheet_name, index=False)

        ws_run = writer.book[sheet_name]

        plot_dir = os.path.join(BASE_OUTPUT_DIR, f"Run_{i}", "plots")
        os.makedirs(plot_dir, exist_ok=True)

        met = res["metrics"]
        cvar = met["cumulative_variance"]
        specificity_data = met["specificity"]
        general_data = met["generalization"]

        compactness_img = _plot_metric_curve(cvar, "Compactness", "Variance", i, plot_dir)
        specificity_img = _plot_metric_curve(specificity_data, "Specificity Error", "Error", i, plot_dir)
        general_img     = _plot_metric_curve(general_data, "Generalization Error", "Error", i, plot_dir)

        row_img1 = 1
        row_img2 = 12
        row_img3 = 23

        # Insertion d'images
        imgA = ExcelImage(compactness_img); imgA.width, imgA.height = 310, 180
        imgB = ExcelImage(specificity_img); imgB.width, imgB.height = 310, 180
        imgC = ExcelImage(general_img);     imgC.width, imgC.height = 310, 180

        ws_run.add_image(imgA, f"A{row_img1}")
        ws_run.add_image(imgB, f"A{row_img2}")
        ws_run.add_image(imgC, f"A{row_img3}")

        # Petits commentaires
        ws_run.cell(row=row_img1+9, column=1, value=f"Composantes pour 95%: {met['compactness_required']}")
        if len(specificity_data) > 0:
            ws_run.cell(row=row_img2+9, column=1, value=f"Specificity final error: {specificity_data[-1]:.4f}")
        if len(general_data) > 0:
            ws_run.cell(row=row_img3+9, column=1, value=f"Generalization final error: {general_data[-1]:.4f}")

        # PC1/PC2
        pc1_pc2_df = pd.DataFrame(res["pca_projection"], columns=["PC1", "PC2"])
        pc1_pc2_df.to_excel(writer, sheet_name=sheet_name, startrow=34, index=False)

    writer.close()
    print(f"Fichier Excel sauvegardé : {excel_filename}")



 ### 5. Script Principal

In [44]:
# 1) Dossier pour grooming
base_output_for_grooming = os.path.join(BASE_OUTPUT_DIR, "GROOMING")
os.makedirs(base_output_for_grooming, exist_ok=True)

OUTPUT_PATH_GROOM = os.path.join(base_output_for_grooming, "OUTPUT")
os.makedirs(OUTPUT_PATH_GROOM, exist_ok=True)
groom_dir = os.path.join(OUTPUT_PATH_GROOM, "groomed")

# 2) Pré-traitement (grooming)
(shape_seg_list,
 shape_filenames,
 dataset_ids,
 shape_names,
 rigid_transforms,
 groomed_files) = run_preprocessing(DATASET_PATHS, SHAPE_EXT, base_output_for_grooming)

# 3) Liste des runs (combinaisons GRID_PARAMS)
grid_keys = list(GRID_PARAMS.keys())
grid_values = [GRID_PARAMS[k] for k in grid_keys]
runs = []
for combo in itertools.product(*grid_values):
    run_dict = {}
    for k, val in zip(grid_keys, combo):
        run_dict[k] = val
    runs.append(run_dict)

# 4) Exécution en boucle
results = []
for i, run_params in enumerate(runs, start=1):
    print("\n========================================")
    print(f"Exécution {i} avec les paramètres :")
    for key, val in run_params.items():
        print(f"  {key} = {val}")

    out = run_optimization_and_analysis(
        run_params,
        i,
        shape_seg_list,
        shape_filenames,
        rigid_transforms,
        groomed_files,
        BASE_OUTPUT_DIR
    )
    results.append(out)

# 5) Export Excel
excel_filename = os.path.join(BASE_OUTPUT_DIR, "grid_search_results.xlsx")
save_results_to_excel(results, excel_filename, grid_keys)

print("\nPipeline terminée.")



----------------------------------------
Step 1. Acquire Data
Nombre de shapes : 4

----------------------------------------
Step 2. Groom - Data Pre-processing


Grooming shapes: 100%|██████████| 4/4 [01:31<00:00, 22.87s/it]



----------------------------------------
Step 3. Groom - Rigid Transformations
Image de référence trouvée : RF_S2_FULGUR_008_181477_label_4


Calcul des transformations: 100%|██████████| 4/4 [01:25<00:00, 21.47s/it]



Exécution 1 avec les paramètres :
  number_of_particles = 16

Lancement Optim+Analyse - Exécution 1

----------------------------------------
Step 4. Optimize - Particle Based Optimization
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_007_48871_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_008_181477_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_009_25705_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_076_46061_left_label_4.vtk
Lancement de l'optimisation via shapeworks...

----------------------------------------
Calcul de l'ACP
Forme des particules aplaties : (4, 48)
Résultats de la PCA sauvegardés dans c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE

...

Exécution 2 terminée en 31.26 secondes

Exécution 3 avec les paramètres :
  number_of_particles = 64

Lancement Optim+Analyse - Exécution 3

----------------------------------------
Step 4. Optimize - Particle Based Optimization
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_007_48871_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_008_181477_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_009_25705_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_076_46061_left_label_4.vtk
Lancement de l'optimisation via shapeworks...

----------------------------------------
Calcul de l'ACP
Forme des particules aplaties : (4, 192)
Résultats de la PCA sauvegardés dans c:\Users\sa

Calcul de la spécificité: 100%|██████████| 3/3 [00:00<00:00,  9.26it/s]


Exécution 3 terminée en 80.49 secondes

Exécution 4 avec les paramètres :
  number_of_particles = 128

Lancement Optim+Analyse - Exécution 4

----------------------------------------
Step 4. Optimize - Particle Based Optimization
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_007_48871_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_008_181477_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_009_25705_label_4.vtk
Writing: c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\GROOMING\OUTPUT\groomed\meshes\RF_S2_FULGUR_076_46061_left_label_4.vtk
Lancement de l'optimisation via shapeworks...

----------------------------------------
Calcul de l'ACP
Forme des particules aplaties : (4, 384)
Résultats de la PCA sauvegardés dans c:\Users\s

Calcul de la spécificité: 100%|██████████| 3/3 [00:00<00:00,  4.27it/s]


Exécution 4 terminée en 188.10 secondes

Sauvegarde des résultats dans le fichier Excel...
Fichier Excel sauvegardé : c:\Users\sacha\Desktop\ECN\0_PROJ_REDEV\SOURCE\CODE\OUTPUT_PIPELINE\grid_search_results.xlsx

Pipeline terminée.
